In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Load datasets with error handling
try:
    purchase_behaviour = pd.read_csv("purchase_behaviour.csv")
    transaction_data = pd.read_csv("transaction_data.csv")
except FileNotFoundError as e:
    print(f"Error: {e}. Please check the file paths.")
    exit()

# Merge datasets with more robust merging
transaction_data.rename(columns={'LYLTY_CARD': 'LYLTY_CARD_NBR'}, inplace=True)
data = transaction_data.merge(purchase_behaviour, on='LYLTY_CARD_NBR', how='inner')

# Handle missing values more carefully
data.dropna(subset=['TOT_SALES', 'TXN_ID', 'LIFESTAGE', 'PREMIUM_CUSTOMER'], inplace=True)

# Identifying top 3 most profitable products with more detailed analysis
top_products = (
    data.groupby('PROD_NAME')
    .agg({
        'TOT_SALES': 'sum', 
        'PROD_QTY': 'sum'
    })
    .sort_values(by='TOT_SALES', ascending=False)
    .head(3)
)
print("Top 3 Most Profitable Products:")
print(top_products)

# Customer Loyalty Analysis with more comprehensive feature engineering
df_loyalty = data.groupby('LYLTY_CARD_NBR').agg({
    'TOT_SALES': 'sum',
    'TXN_ID': 'count',
    'PROD_QTY': 'sum'
}).reset_index()

df_loyalty.columns = ['LYLTY_CARD_NBR', 'TOTAL_SALES', 'TRANSACTION_COUNT', 'TOTAL_QUANTITY']

# Merge with additional customer information
df_loyalty = df_loyalty.merge(
    purchase_behaviour[['LYLTY_CARD_NBR', 'LIFESTAGE', 'PREMIUM_CUSTOMER']], 
    on='LYLTY_CARD_NBR', 
    how='left'
)

# Prepare features for clustering
features_for_clustering = ['TOTAL_SALES', 'TRANSACTION_COUNT', 'TOTAL_QUANTITY']

# Use MinMaxScaler for better scaling (alternative to StandardScaler)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_loyalty[features_for_clustering])

# Determine optimal number of clusters using elbow method
from sklearn.metrics import silhouette_score

max_clusters = 10
silhouette_scores = []
inertias = []

for n_clusters in range(2, max_clusters + 1):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    silhouette_scores.append(silhouette_score(X_scaled, cluster_labels))
    inertias.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(2, max_clusters + 1), inertias, marker='o')
plt.title('Elbow Curve')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')

plt.subplot(1, 2, 2)
plt.plot(range(2, max_clusters + 1), silhouette_scores, marker='o')
plt.title('Silhouette Scores')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.tight_layout()
plt.show()

# Choose optimal number of clusters (you can adjust based on the plot)
optimal_clusters = 3
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42, n_init=10)
df_loyalty['Cluster'] = kmeans.fit_predict(X_scaled)

# Analyze characteristics of most loyal customers
loyal_customers = df_loyalty[df_loyalty['Cluster'] == df_loyalty['Cluster'].mode()[0]]
loyal_segments = loyal_customers.groupby(['LIFESTAGE', 'PREMIUM_CUSTOMER']).size().reset_index(name='Count')
print("\nMost Loyal Customer Segments:")
print(loyal_segments)

# Visualization with more detailed insights
plt.figure(figsize=(12, 6))
sns.barplot(data=loyal_segments, x='LIFESTAGE', y='Count', hue='PREMIUM_CUSTOMER')
plt.title('Most Loyal Customer Segments')
plt.xlabel('Life Stage')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Print cluster characteristics
print("\nCluster Characteristics:")
for cluster in range(optimal_clusters):
    cluster_data = df_loyalty[df_loyalty['Cluster'] == cluster]
    print(f"\nCluster {cluster}:")
    print(cluster_data[features_for_clustering].describe())

Top 3 Most Profitable Products:
                                          TOT_SALES  PROD_QTY
PROD_NAME                                                    
Dorito Corn Chp     Supreme 380g            40352.0      6509
Smiths Crnkle Chip  Orgnl Big Bag 380g      36367.6      6164
Smiths Crinkle Chips Salt & Vinegar 330g    34804.2      6106
